This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

Contributors: 鄭宇伸, 喬彥翔

# [Exercise 2] Regression

## 展示題：房價預測（單變數版本）

我們蒐集到了波士頓郊區的房價資料集（[來源](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/?C=N;O=D)），想要從某城鎮的一個 feature，來預測該城鎮自用住宅的房價中位數（`MEDV`）。

我們將這個資料集的 csv 檔讀入至一個 pandas 的 DataFrame：`df`。資料的各個 column 的意義如下：
- `CRIM`：某城鎮的人均犯罪率
- `ZN`：「超過 25,000 平方呎的住宅用地區塊」所佔的比例
- `INDUS`：某城鎮「非零售的商業用地」比例（英畝）
- `NOX`：一氧化氮濃度（以 10 ppm 為單位）
- `RM`：平均每戶有幾個房間
- `AGE`： 1940 年之前所建的房屋，屋主自用的比例
- `DIS`：到波士頓五個就業服務中心的（加權）距離
- `RAD`：使用高速公路的方便性 / 可達性指數
- `TAX`：「總價 / 房屋稅」的比例（單位：10,000 美金）
- `PTRATIO`：某城鎮的「生 / 師」比
- `MEDV`：自用住宅的房價中位數（單位：1,000 美金）




In [ ]:
!pip install numpy pandas matplotlib scikit-learn mlxtend==0.18.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mlxtend.plotting import scatterplotmatrix
from mlxtend.plotting import heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

### Exploratory Data Analysis (EDA)

在開使訓練一個機器學習模型之前，一個很重要的工作是做 Exploratory Data Analysis (EDA)，透過視覺化的方式來了解資料的分佈、feature 之間的關係等。

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/AINTUT/code_2021/main/datasets/"
    "house_pricing.csv",
)

print(df.head())
print(list(df))

In [ ]:
columns = [
    "CRIM",
    "NOX",
    "RM",
    "DIS",
    "TAX",
    "PTRATIO",
    "MEDV",
]

scatterplotmatrix(
    df[columns].to_numpy(),
    figsize=(10, 8),
    names=columns,
    alpha=0.5,
)

plt.tight_layout()
plt.show()

In [ ]:
cm = np.corrcoef(df[columns].to_numpy().T)
hm = heatmap(cm, row_names=columns, column_names=columns)

plt.show()

In [ ]:
df.plot.scatter(x="RM", y="MEDV")
plt.show()

In [ ]:
x_data = df[["RM"]].to_numpy()
y_data = df["MEDV"].to_numpy()

### Data Preprocessing

選定 feature 後，我們將對資料做以下處理：
1. 將資料分割成 training data 與 testing data
2. 用 standardization 對資料做 feature scaling

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.2,
    random_state=0,
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
scaler = StandardScaler()

x_train_std = scaler.fit_transform(x_train)
x_test_std = scaler.transform(x_test)

print(x_train[:10])
print(x_train_std[:10])

### Training

開始我們訓練模型的流程，主要包含以下步驟：
1. 定義 function set：$f(x) = b + w_{1}x_{1}$
2. 定義 loss function：$L(b, w_{1}) = \frac{1}{2}\sum_{i=1}^{n}(\hat{y}^{(i)} - (b + w_{1}{x}^{(i)}_{1}))^2$
3. 學習，使用 Gradient Descent 來找最佳的 function：$f^{*}$

In [ ]:
def predict(x, weights):
    return np.dot(x, weights[1:]) + weights[0]

In [ ]:
def calculate_loss(y_gt, y_pred):
    loss = ((y_gt - y_pred) ** 2).sum() / 2.0

    return loss

In [ ]:
def fit(x_train, y_train, epoches, learning_rate):
    weights = np.zeros(x_train.shape[1] + 1)
    losses = []

    for _ in range(epoches):
        y_pred = predict(x_train, weights)

        diff = y_train - y_pred
        weights[0] = weights[0] - learning_rate * -diff.sum()
        weights[1:] = weights[1:] - learning_rate * -x_train.T.dot(diff)

        losses.append(calculate_loss(y_train, y_pred))

    return weights, losses

In [ ]:
epoches = 20
learning_rate = 0.001

weights, losses = fit(x_train_std, y_train, epoches, learning_rate)

In [ ]:
plt.plot(range(1, epoches + 1), losses)
plt.ylabel("SSE")
plt.xlabel("Epoch")
plt.show()

### Evaluation

來看一下我們的訓練成果！

In [ ]:
def reg_plot(x, y_gt, y_pred):
    plt.scatter(x, y_gt, c="steelblue", edgecolor="white")
    plt.plot(x, y_pred, c="black")

    plt.xlabel("RM")
    plt.ylabel("MEDV")

    plt.show()

In [ ]:
y_pred = [predict(x, weights) for x in x_train_std]

reg_plot(x_train, y_train, y_pred)

mse = mean_squared_error(y_train, y_pred)

print("MSE of Training Data: {}".format(mse))

In [ ]:
y_pred = [predict(x, weights) for x in x_test_std]

reg_plot(x_test, y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

print("MSE of Testing Data: {}".format(mse))

### 改用 Scikit-Learn 做 Training

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_train)

reg_plot(x_train, y_train, y_pred)

mse = mean_squared_error(y_train, y_pred)

print("MSE of Training Data: {}".format(mse))

In [ ]:
y_pred = lr.predict(x_test)

reg_plot(x_test, y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

print("MSE of Testing Data: {}".format(mse))